## Creating CNN Using Scratch And Transfer Learning

Please download the dataset from the below url

In [ ]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.vgg19 import VGG19
#from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
mobilenet = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top = False,
    pooling = 'avg',
    weights = 'imagenet',
    input_shape = (224, 224, 3)
)

mobilenet.summary()

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'cell_images/Train'
valid_path = 'cell_images/Test'


In [3]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

#mobilnet = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


#mobilnet.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
# don't train existing weights
for layer in mobilenet.layers:
    layer.trainable = False

In [5]:
  # useful for getting number of output classes
folders = glob('cell_images/train/*')

In [6]:
folders

['cell_images/train\\Parasitized', 'cell_images/train\\Uninfected']

In [7]:
# our layers - you can add more if you want
x = Flatten()(mobilenet.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=mobilenet.input, outputs=prediction)

In [9]:
from tensorflow.keras.layers import MaxPooling2D

In [10]:
'''### Create Model from scratch using CNN
outputs=Sequential()
outputs.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(224,224,3)))
outputs.add(MaxPooling2D(pool_size=2))
outputs.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))
outputs.add(MaxPooling2D(pool_size=2))
outputs.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
outputs.add(MaxPooling2D(pool_size=2))
outputs.add(Flatten())
outputs.add(Dense(500,activation="relu"))
outputs.add(Dense(2,activation="softmax"))
outputs.summary()'''

'### Create Model from scratch using CNN\noutputs=Sequential()\noutputs.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(224,224,3)))\noutputs.add(MaxPooling2D(pool_size=2))\noutputs.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))\noutputs.add(MaxPooling2D(pool_size=2))\noutputs.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))\noutputs.add(MaxPooling2D(pool_size=2))\noutputs.add(Flatten())\noutputs.add(Dense(500,activation="relu"))\noutputs.add(Dense(2,activation="softmax"))\noutputs.summary()'

In [11]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [13]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [23]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('cell_images/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 27558 images belonging to 2 classes.


In [24]:
training_set

In [25]:
test_set = test_datagen.flow_from_directory('cell_images/test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 184 images belonging to 2 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=2,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1723 steps, validate for 12 steps
Epoch 1/2
 875/1723 [==============>...............] - ETA: 7:23 - loss: 0.4205 - accuracy: 0.8212

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_vgg19.h5')

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_vgg19.h5')

In [ ]:
img=image.load_img('cell_images/test/Uninfected/2.png',target_size=(224,224))



In [ ]:
x=image.img_to_array(img)
x

In [ ]:
x.shape

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
if(a==1):
    print("Uninfected")
else:
    print("Infected")